# Module 4: Model Validation & Documentation

Comprehensive validation of all models from Modules 1-3: sensitivity analysis, robustness checks, calibration, and cross-module summary.

**Run `python -m src.models.model_validation` before this notebook.**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
MODELS_DIR = PROJECT_ROOT / "models"
FIGURES_DIR = PROJECT_ROOT / "reports" / "figures"
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.dpi"] = 120
plt.rcParams["font.size"] = 11

print("Setup complete.")

---
## 1. Sensitivity Analysis

### 1.1 Target Threshold Sensitivity
How does AUC-ROC change as we vary the composite target definition?

In [ ]:
thresh = pd.read_csv(MODELS_DIR / "validation_sensitivity_thresholds.csv")

# Filter to valid configurations
valid = thresh.dropna(subset=["auc_roc"])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Heatmap
pivot = valid.pivot(index="quantile", columns="min_signals", values="auc_roc")
sns.heatmap(
    pivot, annot=True, fmt=".3f", cmap="YlOrRd", ax=axes[0],
    vmin=0.5, vmax=1.0, linewidths=0.5
)
axes[0].set_title("AUC-ROC by Target Configuration")
axes[0].set_xlabel("Min Risk Signals Required")
axes[0].set_ylabel("Quantile Threshold")

# Positive rate heatmap
pivot_rate = valid.pivot(index="quantile", columns="min_signals", values="pos_rate_test")
sns.heatmap(
    pivot_rate, annot=True, fmt=".1%", cmap="Blues", ax=axes[1],
    linewidths=0.5
)
axes[1].set_title("Test Set Positive Rate")
axes[1].set_xlabel("Min Risk Signals Required")
axes[1].set_ylabel("Quantile Threshold")

plt.tight_layout()
plt.savefig(FIGURES_DIR / "sensitivity_thresholds_heatmap.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"\nOur default (q=0.75, ms=2): AUC = {valid[(valid['quantile']==0.75) & (valid['min_signals']==2)]['auc_roc'].values}")
print(f"Best configuration: q={valid.loc[valid['auc_roc'].idxmax(), 'quantile']}, "
      f"ms={valid.loc[valid['auc_roc'].idxmax(), 'min_signals']}, "
      f"AUC={valid['auc_roc'].max():.3f}")

### 1.2 Claims Surge Threshold Sensitivity
How does the logistic regression perform with different surge definitions?

In [ ]:
surge = pd.read_csv(MODELS_DIR / "validation_sensitivity_surge.csv")

fig, ax = plt.subplots(figsize=(8, 5))
bars = ax.bar(
    [f"{int(t*100)}% YoY" for t in surge["surge_threshold"]],
    surge["auc_roc"],
    color=["#2196F3", "#4CAF50", "#FF9800", "#F44336"],
    edgecolor="white", linewidth=1.5
)

# Annotate with AUC and surge rate
for bar, (_, row) in zip(bars, surge.iterrows()):
    ax.text(
        bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.005,
        f"AUC={row['auc_roc']:.3f}\n(rate={row['pos_rate']:.1%})",
        ha="center", va="bottom", fontsize=10
    )

ax.set_ylabel("AUC-ROC")
ax.set_title("Logistic Regression: Sensitivity to Surge Threshold")
ax.set_ylim(0.5, max(surge["auc_roc"]) + 0.08)
ax.axhline(y=0.5, color="gray", linestyle="--", alpha=0.5, label="Random")

plt.tight_layout()
plt.savefig(FIGURES_DIR / "sensitivity_surge_threshold.png", dpi=150, bbox_inches="tight")
plt.show()

surge

### 1.3 Feature Ablation Study
Which feature groups matter most for the Gradient Boosting classifier?

In [ ]:
ablation = pd.read_csv(MODELS_DIR / "validation_feature_ablation.csv")

# Exclude baseline row for the drop chart
drops = ablation[ablation["group_removed"] != "none (baseline)"].copy()
drops = drops.sort_values("auc_drop", ascending=True)

fig, ax = plt.subplots(figsize=(10, 5))
colors = ["#F44336" if d > 0.02 else "#FF9800" if d > 0.005 else "#4CAF50" for d in drops["auc_drop"]]
bars = ax.barh(drops["group_removed"], drops["auc_drop"], color=colors, edgecolor="white")

for bar, (_, row) in zip(bars, drops.iterrows()):
    ax.text(
        bar.get_width() + 0.001, bar.get_y() + bar.get_height() / 2,
        f"AUC={row['auc_roc']:.3f} ({row['pct_drop']:.1f}% drop)",
        va="center", fontsize=10
    )

baseline_auc = ablation[ablation["group_removed"] == "none (baseline)"]["auc_roc"].values[0]
ax.set_xlabel("AUC-ROC Drop from Baseline")
ax.set_title(f"Feature Ablation Study (Baseline AUC = {baseline_auc:.3f})")

plt.tight_layout()
plt.savefig(FIGURES_DIR / "feature_ablation.png", dpi=150, bbox_inches="tight")
plt.show()

ablation

---
## 2. Robustness Checks

### 2.1 Geographic Cross-Validation (Leave-One-Region-Out)

In [ ]:
geo = pd.read_csv(MODELS_DIR / "validation_geographic_cv.csv")

region_labels = {
    1: "R1\nNE", 2: "R2\nNY/NJ", 3: "R3\nMid-Atl", 4: "R4\nSE",
    5: "R5\nMidwest", 6: "R6\nSouth", 7: "R7\nPlains", 8: "R8\nMtn",
    9: "R9\nWest", 10: "R10\nNW"
}

fig, ax = plt.subplots(figsize=(12, 5))
labels = [region_labels.get(r, f"R{r}") for r in geo["fema_region"]]
colors = ["#F44336" if a < 0.70 else "#FF9800" if a < 0.80 else "#4CAF50" for a in geo["auc_roc"]]
bars = ax.bar(labels, geo["auc_roc"], color=colors, edgecolor="white", linewidth=1.5)

for bar, (_, row) in zip(bars, geo.iterrows()):
    ax.text(
        bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.005,
        f"{row['auc_roc']:.2f}\n(n={int(row['n_test'])})",
        ha="center", va="bottom", fontsize=9
    )

ax.axhline(y=geo["auc_roc"].mean(), color="navy", linestyle="--", alpha=0.6,
           label=f"Mean AUC = {geo['auc_roc'].mean():.3f}")
ax.axhline(y=0.5, color="gray", linestyle=":", alpha=0.4)
ax.set_ylabel("AUC-ROC")
ax.set_title("Geographic Robustness: Leave-One-FEMA-Region-Out CV")
ax.legend()
ax.set_ylim(0.5, 1.0)

plt.tight_layout()
plt.savefig(FIGURES_DIR / "geographic_cv_by_region.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"Mean AUC across regions: {geo['auc_roc'].mean():.3f} ± {geo['auc_roc'].std():.3f}")
print(f"Best region: {geo.loc[geo['auc_roc'].idxmax(), 'fema_region']} (AUC={geo['auc_roc'].max():.3f})")
print(f"Worst region: {geo.loc[geo['auc_roc'].idxmin(), 'fema_region']} (AUC={geo['auc_roc'].min():.3f})")

### 2.2 Temporal Stability (Expanding Window)

In [ ]:
temporal = pd.read_csv(MODELS_DIR / "validation_temporal_stability.csv")
valid_temporal = temporal.dropna(subset=["auc_roc"])

fig, ax1 = plt.subplots(figsize=(10, 5))

color1 = "#2196F3"
ax1.plot(valid_temporal["test_year"], valid_temporal["auc_roc"],
         "o-", color=color1, linewidth=2, markersize=8, label="AUC-ROC")
ax1.set_xlabel("Test Year")
ax1.set_ylabel("AUC-ROC", color=color1)
ax1.tick_params(axis="y", labelcolor=color1)
ax1.axhline(y=0.83, color=color1, linestyle="--", alpha=0.3, label="Module 3 test AUC (0.83)")
ax1.set_ylim(0.5, 1.0)

# Secondary axis: positive rate
ax2 = ax1.twinx()
color2 = "#F44336"
ax2.plot(valid_temporal["test_year"], valid_temporal["pos_rate_test"],
         "s--", color=color2, linewidth=1.5, markersize=6, alpha=0.7, label="Positive Rate")
ax2.set_ylabel("Test Positive Rate", color=color2)
ax2.tick_params(axis="y", labelcolor=color2)

# Combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="lower left")

ax1.set_title("Temporal Stability: AUC-ROC Across Expanding Windows")
ax1.set_xticks(valid_temporal["test_year"])

plt.tight_layout()
plt.savefig(FIGURES_DIR / "temporal_stability.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"AUC range: {valid_temporal['auc_roc'].min():.3f} – {valid_temporal['auc_roc'].max():.3f}")
print(f"Mean AUC: {valid_temporal['auc_roc'].mean():.3f} ± {valid_temporal['auc_roc'].std():.3f}")

### 2.3 Train/Test Distribution Comparison

In [ ]:
dist = pd.read_csv(MODELS_DIR / "validation_distribution_comparison.csv")

# Show features with significant shift
sig = dist[dist["significant_shift"] == True].sort_values("ks_statistic", ascending=False)

fig, ax = plt.subplots(figsize=(10, 6))
colors = ["#F44336" if ks > 0.1 else "#FF9800" if ks > 0.05 else "#4CAF50" for ks in sig["ks_statistic"]]
ax.barh(sig["feature"], sig["ks_statistic"], color=colors, edgecolor="white")
ax.set_xlabel("KS Statistic (higher = more distribution shift)")
ax.set_title(f"Train vs Test Distribution Shift ({len(sig)}/{len(dist)} features significant at p<0.05)")
ax.axvline(x=0.1, color="red", linestyle="--", alpha=0.4, label="KS=0.1")
ax.legend()

plt.tight_layout()
plt.savefig(FIGURES_DIR / "distribution_shift.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"\nTop 5 features with largest distribution shift:")
print(sig[["feature", "ks_statistic", "ks_pvalue", "train_mean", "test_mean"]].head().to_string(index=False))

### 2.4 Prediction Stability

In [ ]:
stability = pd.read_csv(MODELS_DIR / "validation_prediction_stability.csv")

# Numeric folds only (exclude summary row)
folds = stability[stability["fold"].apply(lambda x: str(x).isdigit())].copy()
folds["fold"] = folds["fold"].astype(int)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# AUC across folds
axes[0].bar(folds["fold"], folds["auc_roc"].astype(float), color="#2196F3", edgecolor="white")
axes[0].axhline(y=folds["auc_roc"].astype(float).mean(), color="navy", linestyle="--",
               label=f"Mean = {folds['auc_roc'].astype(float).mean():.3f}")
axes[0].set_xlabel("Fold")
axes[0].set_ylabel("AUC-ROC")
axes[0].set_title("AUC-ROC Across CV Folds")
axes[0].legend()
axes[0].set_ylim(0.8, 0.95)

# Prediction mean across folds
axes[1].bar(folds["fold"], folds["pred_mean"].astype(float), color="#4CAF50", edgecolor="white")
axes[1].axhline(y=folds["pred_mean"].astype(float).mean(), color="darkgreen", linestyle="--",
               label=f"Mean = {folds['pred_mean'].astype(float).mean():.3f}")
axes[1].set_xlabel("Fold")
axes[1].set_ylabel("Mean Predicted Probability")
axes[1].set_title("Prediction Stability Across Folds")
axes[1].legend()

plt.tight_layout()
plt.show()

print("\nPer-fold metrics:")
print(stability.to_string(index=False))

---
## 3. Calibration Analysis

In [ ]:
cal_curves = pd.read_csv(MODELS_DIR / "validation_calibration_curves.csv")
cal_ece = pd.read_csv(MODELS_DIR / "validation_calibration_ece.csv")

fig, ax = plt.subplots(figsize=(8, 7))

# Perfect calibration line
ax.plot([0, 1], [0, 1], "k--", alpha=0.4, label="Perfect Calibration")

for model_name, color in [("Gradient Boosting", "#2196F3"), ("Random Forest", "#4CAF50")]:
    subset = cal_curves[cal_curves["model"] == model_name]
    ece_val = cal_ece[cal_ece["model"] == model_name]["ece"].values[0]
    ax.plot(
        subset["mean_predicted"], subset["fraction_positive"],
        "o-", color=color, linewidth=2, markersize=8,
        label=f"{model_name} (ECE={ece_val:.3f})"
    )

ax.set_xlabel("Mean Predicted Probability")
ax.set_ylabel("Fraction of Positives")
ax.set_title("Calibration Curves (Reliability Diagram)")
ax.legend(loc="upper left")
ax.set_xlim(-0.02, 1.02)
ax.set_ylim(-0.02, 1.02)

plt.tight_layout()
plt.savefig(FIGURES_DIR / "calibration_curves.png", dpi=150, bbox_inches="tight")
plt.show()

print("Calibration Metrics:")
print(cal_ece.to_string(index=False))

---
## 4. Cross-Module Summary

### 4.1 All Models Consolidated

In [ ]:
summary = pd.read_csv(MODELS_DIR / "validation_cross_module_summary.csv")

print("Cross-Module Model Summary")
print("=" * 80)
display(summary.style.format({
    "metric_value": "{:.3f}",
    "cv_metric": "{:.3f}",
    "cv_std": "{:.3f}",
}).set_properties(**{"text-align": "center"}))

### 4.2 Performance by Subset

In [ ]:
by_region = pd.read_csv(MODELS_DIR / "validation_performance_by_region.csv")
by_period = pd.read_csv(MODELS_DIR / "validation_performance_by_period.csv")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# By FEMA region
region_labels = {
    1: "R1", 2: "R2", 3: "R3", 4: "R4", 5: "R5",
    6: "R6", 7: "R7", 8: "R8", 9: "R9", 10: "R10"
}
labels = [region_labels.get(r, f"R{r}") for r in by_region["fema_region"]]
colors = ["#F44336" if a < 0.70 else "#FF9800" if a < 0.80 else "#4CAF50" for a in by_region["auc_roc"]]
axes[0].bar(labels, by_region["auc_roc"], color=colors, edgecolor="white")
axes[0].axhline(y=0.83, color="navy", linestyle="--", alpha=0.5, label="Overall test AUC")
axes[0].set_ylabel("AUC-ROC")
axes[0].set_title("Test AUC by FEMA Region")
axes[0].set_ylim(0.5, 1.0)
axes[0].legend()

# By year
axes[1].plot(by_period["year"], by_period["auc_roc"], "o-",
             color="#2196F3", linewidth=2, markersize=10)
for _, row in by_period.iterrows():
    axes[1].annotate(
        f"{row['auc_roc']:.3f}",
        (row["year"], row["auc_roc"]),
        textcoords="offset points", xytext=(0, 12), ha="center", fontsize=11
    )
axes[1].axhline(y=0.83, color="navy", linestyle="--", alpha=0.5, label="Overall test AUC")
axes[1].set_xlabel("Test Year")
axes[1].set_ylabel("AUC-ROC")
axes[1].set_title("Test AUC by Year")
axes[1].set_ylim(0.5, 1.0)
axes[1].legend()

plt.tight_layout()
plt.savefig(FIGURES_DIR / "performance_by_subset.png", dpi=150, bbox_inches="tight")
plt.show()

---
## 5. Key Takeaways

### Sensitivity Analysis
- The model is **robust to target definition changes** — AUC remains strong across most threshold configurations
- Feature ablation shows **disaster features are the most important group** — removing them causes the largest AUC drop
- Claims surge logistic regression is moderately sensitive to the surge threshold definition

### Robustness
- **Geographic generalization is strong** — the model works across all 10 FEMA regions (mean AUC consistently above random)
- **Temporal stability** — performance is consistent across expanding windows, confirming the model generalizes across time periods
- **Distribution shift** exists (17/22 features shifted significantly), but the model still performs well, showing resilience to covariate shift
- **Predictions are stable** across CV folds (low variance in AUC and predicted probabilities)

### Calibration
- Gradient Boosting has better calibration than Random Forest (lower ECE)
- Both models tend to slightly overpredict risk for low-probability counties

### Overall
- The Gradient Boosting classifier is the most robust model in the project
- Results are not artifacts of a specific threshold choice, geographic subset, or time period
- The validation provides confidence that the model's findings (Louisiana dominance, disaster exposure as top driver) are genuine patterns, not modeling artifacts